In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import warnings;
from pysqldf import SQLDF
import pandasql as psql
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import KFold
import sklearn.ensemble as ske
import lightgbm as lgb
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

warnings.filterwarnings('ignore')

In [2]:
train01 = pd.read_csv("C:\\Kaggle\\Cars\\Data\\TrnDataForLGB.csv")
test01 = pd.read_csv("C:\\Kaggle\\Cars\\Data\\TstDataForLGB.csv")

In [5]:
Remove_List = ["id","Price","Name","Lag_Price2_MIN","Lag_Price2_MAX",
               "Lag_Price3_MIN","Lag_Price3_MAX","Lag_Price","Lag_Price3","Engine_Group",
               "Power_Group","TrainTestInd","CarCompName","RateChng1","RateChng2","RateChng3",
               "Lag_Price4_MIN","Lag_Price4_MAX","Lag_Price4_MIN_BY_MAX"]
feature_names = list(set(list(train01.columns)) - set(Remove_List))
feature_names

['CompNameCarName_FABIA',
 'LocationChennai',
 'CompName_FORD',
 'CompNameCarName_FIGO',
 'CompName_AUDI',
 'CompNameCarName_X3',
 'LocationHyderabad',
 'CompNameCarName_PAJERO',
 'CompNameCarName_KWID',
 'CompNameCarName_CRETA',
 'CompNameCarName_DUSTER',
 'CompNameCarName_ETIOS',
 'CompNameCarName_KUV',
 'CompNameCarName_VERNA',
 'CompNameCarName_5',
 'Mileage',
 'CompNameCarName_MOBILIO',
 'Fuel_TypeCNG',
 'CompNameCarName_RITZ',
 'CompNameCarName_OPTRA',
 'CompNameCarName_ACCENT',
 'CompNameCarName_SANTA',
 'CompNameCarName_ALTO',
 'CompNameCarName_XYLO',
 'CompName_JEEP',
 'LocationMumbai',
 'CompNameCarName_CITY',
 'CompNameCarName_WAGON',
 'CompNameCarName_SANTRO',
 'LocationPune',
 'CompName_MERCEDES-BENZ',
 'CompNameCarName_ELANTRA',
 'CompNameCarName_DZIRE',
 'CompNameCarName_BRIO',
 'CompNameCarName_Q5',
 'CompNameCarName_SWIFT',
 'CompNameCarName_EECO',
 'CompNameCarName_BOLERO',
 'CompName_FIAT',
 'CompNameCarName_I10',
 'CompNameCarName_TERRANO',
 'CompName_NISSAN',
 'Loc

In [7]:
train02 = train01.copy(deep=True)
train02.reset_index(drop = True, inplace = True)
kf = KFold(n_splits = 5, shuffle = True, random_state = 100)
kf.get_n_splits(train02)

5

In [33]:
IterationNum = 1
for train_index, test_index in kf.split(train02):
    print("Running CV Iteration Num :", IterationNum)
    MOD_DATA_2_TRAIN, MOD_DATA_2_TEST = train02.iloc[train_index], train02.iloc[test_index]
    
    parameters = {  'objective': 'regression',
                    'metric': 'rmse',
                    'boosting': 'gbdt',
                    'feature_fraction': 0.4,
                    'bagging_fraction': 0.6,
                    'bagging_freq' : 0,
                    'learning_rate': 0.005,
                    'min_data_in_leaf': 2,
                    'max_depth': 4,
                    'seed': 500,
                    'max_bin': 75,
                    'min_data_in_bin': 5,
                    'verbosity': -1,
                    'silent': -1  }
    
    X_TRAIN = pd.DataFrame(MOD_DATA_2_TRAIN[feature_names])
    Y_TRAIN = MOD_DATA_2_TRAIN["Price"]
                
    X_TEST = pd.DataFrame(MOD_DATA_2_TEST[feature_names])
    Y_TEST = MOD_DATA_2_TEST["Price"]
    
    train_data = lgb.Dataset(X_TRAIN,
                             label = Y_TRAIN)
    valid_data = lgb.Dataset(X_TEST,
                             label = Y_TEST)
    lgb_model = lgb.train(parameters,
                          train_data,
                          valid_sets = valid_data,
                          num_boost_round = 10000000,
                          early_stopping_rounds = 400,
                          verbose_eval = 200)
    MOD_DATA_2_TEST['Predicted_Model_Value'] = lgb_model.predict(pd.DataFrame(MOD_DATA_2_TEST[feature_names]))
    
    if(IterationNum == 1):
        CV_SCORED_DATA = MOD_DATA_2_TEST.copy(deep=True)
        CV_SCORED_DATA.reset_index(drop = True, inplace = True)
        test_scored = lgb_model.predict(pd.DataFrame(test01[feature_names]))
    else:
        CV_SCORED_DATA = pd.concat([CV_SCORED_DATA,MOD_DATA_2_TEST])
        CV_SCORED_DATA.reset_index(drop = True, inplace = True)
        test_scored = test_scored + lgb_model.predict(pd.DataFrame(test01[feature_names]))
                    
    IterationNum = IterationNum + 1
    
    print("Test RMSE : ",sqrt(mean_squared_error(MOD_DATA_2_TEST["Price"], MOD_DATA_2_TEST['Predicted_Model_Value'])))
    
print("CV RMSE : ",sqrt(mean_squared_error(CV_SCORED_DATA["Price"], CV_SCORED_DATA['Predicted_Model_Value'])))
#CV RMSE :  0.1426465946675941
#LB: 0.9462

Running CV Iteration Num : 1
Training until validation scores don't improve for 400 rounds.
[200]	valid_0's rmse: 0.381105
[400]	valid_0's rmse: 0.244997
[600]	valid_0's rmse: 0.198507
[800]	valid_0's rmse: 0.179467
[1000]	valid_0's rmse: 0.169416
[1200]	valid_0's rmse: 0.163913
[1400]	valid_0's rmse: 0.160069
[1600]	valid_0's rmse: 0.157185
[1800]	valid_0's rmse: 0.154881
[2000]	valid_0's rmse: 0.152933
[2200]	valid_0's rmse: 0.151392
[2400]	valid_0's rmse: 0.149863
[2600]	valid_0's rmse: 0.148605
[2800]	valid_0's rmse: 0.14734
[3000]	valid_0's rmse: 0.146211
[3200]	valid_0's rmse: 0.145119
[3400]	valid_0's rmse: 0.144296
[3600]	valid_0's rmse: 0.143381
[3800]	valid_0's rmse: 0.142552
[4000]	valid_0's rmse: 0.141794
[4200]	valid_0's rmse: 0.141099
[4400]	valid_0's rmse: 0.14043
[4600]	valid_0's rmse: 0.139806
[4800]	valid_0's rmse: 0.139188
[5000]	valid_0's rmse: 0.138475
[5200]	valid_0's rmse: 0.138043
[5400]	valid_0's rmse: 0.137581
[5600]	valid_0's rmse: 0.137138
[5800]	valid_0's r

[9000]	valid_0's rmse: 0.180704
[9200]	valid_0's rmse: 0.180508
[9400]	valid_0's rmse: 0.180284
[9600]	valid_0's rmse: 0.180083
[9800]	valid_0's rmse: 0.179908
[10000]	valid_0's rmse: 0.179699
[10200]	valid_0's rmse: 0.179516
[10400]	valid_0's rmse: 0.17939
[10600]	valid_0's rmse: 0.17923
[10800]	valid_0's rmse: 0.179083
[11000]	valid_0's rmse: 0.178969
[11200]	valid_0's rmse: 0.17884
[11400]	valid_0's rmse: 0.178718
[11600]	valid_0's rmse: 0.178588
[11800]	valid_0's rmse: 0.178486
[12000]	valid_0's rmse: 0.178388
[12200]	valid_0's rmse: 0.178275
[12400]	valid_0's rmse: 0.178146
[12600]	valid_0's rmse: 0.178069
[12800]	valid_0's rmse: 0.178001
[13000]	valid_0's rmse: 0.177936
[13200]	valid_0's rmse: 0.177883
[13400]	valid_0's rmse: 0.177789
[13600]	valid_0's rmse: 0.177678
[13800]	valid_0's rmse: 0.177567
[14000]	valid_0's rmse: 0.177489
[14200]	valid_0's rmse: 0.177406
[14400]	valid_0's rmse: 0.177335
[14600]	valid_0's rmse: 0.177266
[14800]	valid_0's rmse: 0.177189
[15000]	valid_0's 

[16000]	valid_0's rmse: 0.129686
[16200]	valid_0's rmse: 0.129567
[16400]	valid_0's rmse: 0.129437
[16600]	valid_0's rmse: 0.129367
[16800]	valid_0's rmse: 0.129286
[17000]	valid_0's rmse: 0.129194
[17200]	valid_0's rmse: 0.129102
[17400]	valid_0's rmse: 0.128987
[17600]	valid_0's rmse: 0.128918
[17800]	valid_0's rmse: 0.128818
[18000]	valid_0's rmse: 0.128743
[18200]	valid_0's rmse: 0.128655
[18400]	valid_0's rmse: 0.128555
[18600]	valid_0's rmse: 0.128479
[18800]	valid_0's rmse: 0.128409
[19000]	valid_0's rmse: 0.128317
[19200]	valid_0's rmse: 0.128245
[19400]	valid_0's rmse: 0.128176
[19600]	valid_0's rmse: 0.128096
[19800]	valid_0's rmse: 0.128039
[20000]	valid_0's rmse: 0.127974
[20200]	valid_0's rmse: 0.127921
[20400]	valid_0's rmse: 0.127881
[20600]	valid_0's rmse: 0.127819
[20800]	valid_0's rmse: 0.127762
[21000]	valid_0's rmse: 0.127707
[21200]	valid_0's rmse: 0.127664
[21400]	valid_0's rmse: 0.127617
[21600]	valid_0's rmse: 0.127576
[21800]	valid_0's rmse: 0.127531
[22000]	va

In [35]:
from scipy import stats
test_scored2 = test_scored / 5.0
stats.describe(test_scored2)

DescribeResult(nobs=1234, minmax=(0.3328331518393663, 4.345887503783606), mean=1.9852669520285584, variance=0.5100080643313538, skewness=0.7459720756831557, kurtosis=0.13212260609489102)

In [42]:
test_scored3 = pd.DataFrame({'Price' : np.exp(test_scored2)-1})

In [43]:
test_scored3.head()

,Price
0,2.749518
1,3.057949
2,16.864857
3,4.199543
4,4.612711


In [44]:
CV_SCORED_DATA.to_csv("C:\\Kaggle\\Cars\\CV_Scored\\20190716_LGB01_CVTRAIN_DS.csv",
                      index = False)
test_scored3.to_csv("C:\\Kaggle\\Cars\\Submission\\20190716_LGB01_TEST_DS.csv",
                    index = False)